First, we import all the required libraries and reset the Nest kernel

In [ ]:
import nest
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
nest.ResetKernel()
nest.SetKernelStatus({
    "resolution": 0.1,   # ms
    "print_time": True
})
np.random.seed(42)

We begin creating the post synaptic neuron from the single neuron model. The model consists in a conductance based integrate and fire neuron the receives in total a 1000 synapses, 800 excitatory and 200 inhibitory. 
First we create the neuron with simple STPD rule (before implementing the Vogels'rule)

In [ ]:
#------------Neuron model parameters (from paper)----------------------------

neuron_params = {
    "C_m": 200.0,        # pF (calculated from using tau=20 ms and gL=10 nS)
    "g_L": 10.0,         # nS
    "E_L": -60.0,        # mV
    "V_th": -50.0,       # mV
    "V_reset": -60.0,    # mV
    "t_ref": 5.0,        # ms
    "E_ex": 0.0,         # mV
    "E_in": -80.0        # mV
}

      # Synapse time constants
tau_ex = 5.0    # ms
tau_in = 10.0   # ms





#---------Create neurons -----------------------------------------------------

     # Postsynaptic neuron
post = nest.Create("iaf_cond_alpha",params={**neuron_params, "tau_syn_ex": tau_ex, "tau_syn_in": tau_in})
#nest.GetStatus(post)
     
     # Presynaptic neurons: Poisson generators + Parrot neurons
N_exc = 100 # Numero de neuronas excitatorias
N_inh = 25  # Numero de neuoronas inhibitorias
rate_exc = 10.0  # Hz numero de pulsos por segundo
rate_inh = 10.0  # Hz 

exc_inputs = nest.Create("poisson_generator", N_exc, params={"rate": rate_exc})
inh_inputs = nest.Create("poisson_generator", N_inh, params={"rate": rate_inh})
   
   # Crear neuronas parrot
exc_parrots = nest.Create("parrot_neuron", N_exc)
inh_parrots = nest.Create("parrot_neuron", N_inh)

   # Conectar Poisson a parrots
nest.Connect(exc_inputs, exc_parrots)
nest.Connect(inh_inputs, inh_parrots)

In [ ]:
# ------ Synaptic connections of excitatory neurons (static) ------
w_exc = 3.0    # nS
delay = 1.0   # ms
nest.Connect(exc_inputs, post, syn_spec={"synapse_model": "static_synapse", "weight": w_exc, "delay": delay})

# ------ Synaptic connections of inhibitory neurons (STPD rule) ------
inh_syn_params = {
    "synapse_model": "vogels_sprekeler_synapse",
    "weight": 0.5,
    "delay": delay,
    "eta": 1e-4,
    "alpha": 0.12,
    "tau": 20.0,
    "Wmax": 300.0
}

nest.Connect(inh_parrots, post, syn_spec=inh_syn_params)
conns = nest.GetConnections(inh_parrots, post)

# ------- Verify weight values ----------------------------------------------
weights = [c["weight"] for c in nest.GetStatus(conns)]
print("Peso inhibitorio promedio:", np.mean(weights))
print("Peso inhibitorio min/max:", np.min(weights), np.max(weights))


# ------- Simulate -----------------------------------------------------------
nest.Simulate(1000.0)

# ------- Verify weight values after simulation ------------------------------
weights = [c["weight"] for c in nest.GetStatus(conns)]
print("Peso inhibitorio promedio:", np.mean(weights))
print("Peso inhibitorio min/max:", np.min(weights), np.max(weights))

After implementing the simple rule, let's modulate the signals from the poisson generators

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({
    "resolution": 0.5,   # ms
    "print_time": True
})

#-------------Synapse time constants--------------------------

tau_ex = 5.0    # ms
tau_in = 10.0   # ms

#---------Create neurons ----------

     # Postsynaptic neuron
post = nest.Create("iaf_cond_alpha",params={**neuron_params, "tau_syn_ex": tau_ex, "tau_syn_in": tau_in})
    # Presynaptic neurons  
N_exc = 100 # Number of excitatory neurons
N_inh = 25  # Number of inhibitory neurons
rate_exc = 10.0  # Hz number of pulses per second
rate_inh = 10.0  # Hz 
   # Parrot neurons 
exc_parrots = nest.Create("parrot_neuron", N_exc)
inh_parrots = nest.Create("parrot_neuron", N_inh)


#----------- Senal modulada en tau = 50 --------
T_sim = 1000.0
dt_signal = 5.0
tau_signal = 50.0
times = np.arange(dt_signal, T_sim + dt_signal, dt_signal)
noise = np.random.randn(len(times))
signal = np.zeros_like(noise)
alpha = dt_signal / tau_signal

for i in range(1, len(signal)):
    signal[i] = signal[i-1] + alpha * (-signal[i-1] + noise[i])

rate_mean = 10.0
rate_std = 5.0
rates = rate_mean + rate_std * signal
rates[rates < 0.1] = 0.1


#--------------Creear generadores de Poisson modulados -------------
exc_gen = nest.Create(
    "inhomogeneous_poisson_generator",
    params={
        "rate_times": times.tolist(),
        "rate_values": rates.tolist()
    }
)

inh_gen = nest.Create(
    "inhomogeneous_poisson_generator",
    params={
        "rate_times": times.tolist(),
        "rate_values": rates.tolist()
    }
)
#-----------Conectar generadores modulados a parrots(neurons) -------
nest.Connect(exc_gen, exc_parrots)
nest.Connect(inh_gen, inh_parrots)


# ------ Sinapsis de neuronas exitartorias (estaticas)------
w_exc = 3.0    # nS
delay = 1.0   # ms
nest.Connect(exc_parrots, post, syn_spec={"synapse_model": "static_synapse", "weight": w_exc, "delay": delay})

# ------ Sinapsis de neuronas inhibitorias (STPD rule)------
inh_syn_params = {
    "synapse_model": "vogels_sprekeler_synapse",
    "weight": 0.5,
    "delay": delay,
    "eta": 1e-4,
    "alpha": 0.12,
    "tau": 20.0,
    "Wmax": 300.0
}
nest.Connect(inh_parrots, post, syn_spec=inh_syn_params)


conns = nest.GetConnections(inh_parrots, post)
weights = [c["weight"] for c in nest.GetStatus(conns)]
print("Peso inhibitorio promedio:", np.mean(weights))
print("Peso inhibitorio min/max:", np.min(weights), np.max(weights))
